In [ ]:
import pandas as pd
patients = pd.read_csv("Patient Matching Data.csv")

In [ ]:
patients.head()

In [ ]:
cleanPatients = patients.drop(["MI"], axis =1)
cleanPatients.head()

In [ ]:
def validateAcc(acc):
    if len(str(acc)) != 11:
        return ''
    else:
        if acc[9] != "-":
            return ''
        else:
            return acc

cleanPatients["Patient Acct #"] = cleanPatients["Patient Acct #"].apply(validateAcc)


In [ ]:
cleanPatients.head(30)

In [ ]:
tokenized_name = cleanPatients['First Name'].apply(lambda x: x.split())

tokenized_name.head()

In [ ]:
from nltk import PorterStemmer

ps = PorterStemmer()

tokenized_name = tokenized_name.apply(lambda x: [ps.stem(i) for i in x])

tokenized_name.head()

In [ ]:
tokenized_name.head(40)

In [ ]:
 def lowerstr(string):
        return(str(string).lower())
    
cleanPatients["First Name"] = cleanPatients['First Name'].apply(lowerstr)
cleanPatients["Last Name"] = cleanPatients['Last Name'].apply(lowerstr)

cleanPatients.head()

In [ ]:
# Maps letters to phonetic groups
soundex_map = { 
    **dict.fromkeys("BFPV", '1'),
    **dict.fromkeys("CGJKQSXZ", '2'),
    **dict.fromkeys("DT", '3'),
    **dict.fromkeys("L", '4'),
    **dict.fromkeys("MN", '5'),
    **dict.fromkeys("R", '6'),
    **dict.fromkeys("AEIOUHWY", '.'),
}

soundex_map["J"]

In [ ]:
def get_soundex(token):
    """Get the soundex code for the string"""
    token = token.upper()

    # first letter of input is always the first letter of soundex
    soundex = token[0]
    
    for char in token[1:]:
            code = soundex_map[char]
            if code != soundex[-1]:
                soundex += code

    # remove vowels and 'H', 'W' and 'Y' from soundex
    soundex = soundex.replace(".", "")
    
    # trim or pad to make soundex a 4-character code
    soundex = soundex[:4].ljust(4, "0")
        
    return soundex

get_soundex("Maine")

In [ ]:
# https://gist.github.com/rogerallen/1583593
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

abbrev_us_state = dict(map(reversed, us_state_abbrev.items()))

In [ ]:
get_soundex("Main")

In [ ]:
print(get_soundex("Sutton"), get_soundex("Suttin"))

In [ ]:
def convert_state(state):
    if state in us_state_abbrev:
        return us_state_abbrev[state]
    
    return state

In [ ]:
def convert_state_code(state):
    if state in abbrev_us_state:
        return abbrev_us_state[state]
    
    return state

cleanPatients["Current State"] = cleanPatients["Current State"].apply(convert_state_code)
cleanPatients["Previous State"] = cleanPatients["Previous State"].apply(convert_state_code)

In [ ]:
def convert_state(state):
    if state in us_state_abbrev:
        return us_state_abbrev[state]
    
    return state

cleanPatients["Current State"] = cleanPatients["Current State"].apply(convert_state)
cleanPatients["Previous State"] = cleanPatients["Previous State"].apply(convert_state)

In [ ]:
cleanPatients["Current State"].value_counts()

In [ ]:
cleanPatients["Previous State"].value_counts()

In [ ]:
cleanPatients['Current Street 2'].value_counts()

In [ ]:
import re
def convert_address_2(address):
    address = str(address).lower()
    apt_no = re.match("(apt|apartment)?\s*#?\s*(\d+)", address)
    
    return apt_no.group(2) if apt_no else ''

cleanPatients['Current Street 2'] = cleanPatients['Current Street 2'].apply(convert_address_2)
cleanPatients['Current Street 2'].value_counts()

In [ ]:
cleanPatients["First Name"].value_counts()

In [ ]:
cleanPatients['Previous Street 2'] = cleanPatients['Previous Street 2'].apply(convert_address_2)
cleanPatients['Previous Street 2'].value_counts()

In [ ]:
street_abv = {
    "ave": "avenue",
    "ln": "lane",
    "st": "street",
    "dr": "drive",
    "rd": "road",
    "cir": "circle",
    "pt": "point",
    "pkway": "parkway",
    "ct": "court",
}

In [ ]:
import string
def clean_address(address):
    address = str(address).lower()
    table = str.maketrans(dict.fromkeys(string.punctuation))
    
    address = address.translate(table)
    
    if address == 'nan':
        return ''
    
    tokens = address.split()
    if len(tokens) >= 3:
        if tokens[-1] in street_abv:
            tokens[-1] = street_abv[tokens[-1]]
        return " ".join(tokens)
    
    return address

cleanPatients["Current Street 1"] = cleanPatients["Current Street 1"].apply(clean_address)
print(cleanPatients["Current Street 1"].value_counts().to_string())

In [ ]:
cleanPatients["Previous Street 1"] = cleanPatients["Previous Street 1"].apply(clean_address)
print(cleanPatients["Previous Street 1"].value_counts().to_string())

In [ ]:
def clean_sex(sex):
    sex = str(sex).lower()
    
    if sex and sex[0] == 'm':
        return 'm'
    elif sex and sex[0] == 'f':
        return 'f'
    
    return ''

In [ ]:
cleanPatients["Sex"] = cleanPatients["Sex"].apply(clean_sex)

cleanPatients["Sex"].value_counts()

In [ ]:
# Maps letters to phonetic groups
soundex_map = { 
    **dict.fromkeys("BFPV", '1'),
    **dict.fromkeys("CGJKQSXZ", '2'),
    **dict.fromkeys("DT", '3'),
    **dict.fromkeys("L", '4'),
    **dict.fromkeys("MN", '5'),
    **dict.fromkeys("R", '6'),
    **dict.fromkeys("AEIOUHWY", '.'),
}

soundex_map["J"]

In [ ]:
def get_soundex(token):
    """Get the soundex code for the string"""
    token = token.upper()

    # first letter of input is always the first letter of soundex
    soundex = token[0]
    
    for char in token[1:]:
            code = soundex_map[char]
            if code != soundex[-1]:
                soundex += code

    # remove vowels and 'H', 'W' and 'Y' from soundex
    soundex = soundex.replace(".", "")
    
    # trim or pad to make soundex a 4-character code
    soundex = soundex[:4].ljust(4, "0")
        
    return soundex

get_soundex("Maine")

In [ ]:
print(get_soundex("Mervin"))
print(get_soundex("Mikel"))
print(get_soundex("Mikel"))
print(get_soundex("Michael"))
print(get_soundex("Mike"))

In [ ]:
soundex_map["Z"]

In [ ]:

from metaphone import doublemetaphone
from enum import Enum

class Threshold(Enum):
    WEAK = 0
    NORMAL = 1
    STRONG = 2

def double_metaphone(value):
    #print(doublemetaphone(value))
    return doublemetaphone(value)

#(Primary Key = Primary Key) = Strongest Match
#(Secondary Key = Primary Key) = Normal Match
#(Primary Key = Secondary Key) = Normal Match
#(Alternate Key = Alternate Key) = Minimal Match
def double_metaphone_compare(tuple1,tuple2,threshold):
    if threshold == Threshold.WEAK:
        if tuple1[1] == tuple2[1]:
            return True
    elif threshold == Threshold.NORMAL:
        if tuple1[0] == tuple2[1] or tuple1[1] == tuple2[0]:
            return True
    else:
        if tuple1[0] == tuple2[0]:
            return True
    return False

In [ ]:
def getMetaphone(string):
    met = double_metaphone(string)
    return met[0]
cleanPatients["metFirst"] = cleanPatients["First Name"].apply(getMetaphone)

In [ ]:
cleanPatients.head()

In [ ]:
cleanPatients.head()

In [ ]:
cleanPatients["names"] = cleanPatients['First Name'].fillna('') + ' ' + cleanPatients['Last Name'].fillna('')
cleanPatients["names"] = cleanPatients['names'].apply(lambda x: x.strip())

In [ ]:
from math import isnan
def convert_zip(z):
    if isnan(z):
        return ''
    
    return str(int(z))

    
cleanPatients['Current Zip Code'] = cleanPatients['Current Zip Code'].apply(convert_zip)
cleanPatients['Current Zip Code'].head()

In [ ]:
cleanPatients["address"] = cleanPatients['Current Street 1'].fillna('') + ' ' + cleanPatients['Current Street 2'] + ' '\
    + cleanPatients['Current City'].fillna('') + ' ' + cleanPatients['Current State'].fillna('') + ' '\
    + cleanPatients['Current Zip Code']
cleanPatients["address"] = cleanPatients["address"].apply(lowerstr)
cleanPatients["address"] = cleanPatients['address'].apply(lambda x: x.strip())
cleanPatients.head(70)

In [ ]:
def clean_dob(date):
    return date.replace('/', '')

cleanPatients['Date of Birth'] = cleanPatients['Date of Birth'].apply(clean_dob)
cleanPatients['Date of Birth'].head()

In [ ]:
from fuzzywuzzy import fuzz

import numpy as np
def compare(a,b):
    scores = []
    count = 0
    if a["names"] and b["names"]:
        a_met = double_metaphone(a["names"])
        b_met = double_metaphone(b["names"])
        
        name_scores = []
        if double_metaphone_compare(a_met, b_met, Threshold.STRONG):
            name_scores.append(100)
        
        name_scores.append(fuzz.token_sort_ratio(a["names"], b["names"]))
        scores.append(max(name_scores))
    
    
        
    if b["address"] == "":
        addressScore = 50
        scores.append(addressScore)
    elif a["address"] and b["address"]:
        addressScore = fuzz.token_sort_ratio(a["address"], b["address"])
        if addressScore > 80:
            scores.append(addressScore)
        elif a["Current Street 1"] and b["Current Street 1"]:
            scores.append(min(addressScore - 20 ,fuzz.token_sort_ratio(a["Current Street 1"], b["Current Street 1"])))
   
        
    
    if a["Sex"] and b["Sex"]:
        if a["Sex"] == b["Sex"]:
            scores.append(90)
        else:
            scores.append(60)
        
    if a["Date of Birth"] and b["Date of Birth"]:
        date_score = fuzz.token_sort_ratio(a["Date of Birth"], b["Date of Birth"])
        if (date_score < 60):
            date_score = 0
    
    
        scores.append(date_score)
        
#     if a["Patient Acct #"] and b["Patient Acct #"]:
#         scores.append(fuzz.token_sort_ratio(a["Patient Acct #"], b["Patient Acct #"])

#     if(len(scores) < 50):
    print(scores)    
    return np.mean(scores)



In [ ]:
fuzz.token_sort_ratio('4211950', '3191960')

In [ ]:


cleanPatients["assigned"] = 0
cleanPatients["groupno"] = -1
groups = []
print(groups)
for index, row in cleanPatients.iterrows():
    if row['assigned'] == 1:
        continue
    else:
        for group_index, x in enumerate(groups):
            temp = compare(cleanPatients.loc[x[0]], row)
            if len(x)>1:
                temp = max(temp, compare(cleanPatients.loc[x[1]], row))
            #print(f'fuzz for {x[0]} and {row["names"]} = {temp},  index is {index}')
            if(temp > 86):
                cleanPatients.loc[index,"groupno"] = group_index+1
                groups[group_index].append(index)
                break
        else:
            groups.append([index])
            cleanPatients.loc[index,"groupno"] = len(groups)

In [ ]:

groups

In [ ]:
fuzz.token_sort_ratio("may field","field may")

In [ ]:
cleanPatients.head(191)


In [ ]:
true_groups = []
for i in range(1,66):
    current_group = []
    for index, row in cleanPatients.loc[cleanPatients['GroupID'] == i].iterrows():
        current_group.append(index)
    true_groups.append(current_group)

for i, true_group in enumerate(true_groups):
    print(true_group, groups[i])



In [ ]:
for i, true_group in enumerate(true_groups):
    if true_group not in groups:
        if i < len(groups):
            print(true_group, groups[i])
        else:
            print(true_group)